In [1]:
from PyQt5 import QtWidgets, QtGui, QtWidgets , uic
from PyQt5.QtWidgets import QGraphicsScene, QGraphicsView, QMessageBox
from PyQt5.QtGui import QPixmap
from PIL import Image
import PIL.ImageQt
from gui import Ui_MainWindow
import tensorflow as tf 
from tensorflow import keras as ks
import numpy as np
import sys
import cv2
import os

# Load Datasets

In [2]:
def load_images_from_folder(folder):
    """
    Loads each image in the given folder and save it in an array.
    
    In case of training and validation data, it saves 3 copies of images:
    original, flipped and rotated copies.
    
    Returns array of images and array of their labels.
    """
    images = []
    labels = []
    classes = 0
    flag = 1
    for file_name in os.listdir(folder):
        if folder == 'Testing': flag = 0 
        sub_dir_path = folder + '/' + file_name
        for filename in os.listdir(sub_dir_path):     
            try:
                img = cv2.imread(os.path.join(sub_dir_path, filename), cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (28, 28))
                flipped_img = cv2.flip(img, 0)
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                
            except Exception as e:
                print(' ')
            
            if img is not None:
                images.append(img)
                labels.append(classes)
                if flag:
                    images.append(flipped_img)
                    images.append(rotated_img)
                    labels.append(classes)
                    labels.append(classes)

        classes += 1
    return np.array(images), np.array(labels)

In [3]:
train_images, train_labels = load_images_from_folder("Training")
validation_images, validation_labels = load_images_from_folder("Validation")
test_images, test_labels = load_images_from_folder("Testing")
class_names = np.array(["actin", "dna", "endosome", "er", "golgia", "golgpp", "lysosome", "microtubulus", "mitochondria", "nucleolus"])
print(len(train_images),len(validation_images), len(test_images))

 
 
 
1818 375 131


In [4]:
print('Training Dataset Shape:{} '.format(train_images.shape)) 
print('No. of Training Dataset Labels:{} '.format(len(train_labels))) 
print('Validation Dataset Shape:{} '.format(validation_images.shape))
print('No. of Validation Dataset Labels:{} '.format(len(validation_labels)))
print('Test Dataset Shape:{} '.format(test_images.shape))
print('No. of Test Dataset Labels:{} '.format(len(test_labels)))

Training Dataset Shape:(1818, 28, 28) 
No. of Training Dataset Labels:1818 
Validation Dataset Shape:(375, 28, 28) 
No. of Validation Dataset Labels:375 
Test Dataset Shape:(131, 28, 28) 
No. of Test Dataset Labels:131 


In [5]:
np.unique(train_images)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [6]:
train_images = train_images / 255.0 
validation_images = validation_images / 255.0
test_images = test_images / 255.0

In [7]:
# Reshape arrays to 4 dims to pass them to our model

train_images = train_images.reshape((1818, 28, 28, 1))
validation_images = validation_images.reshape((375, 28, 28, 1))
test_images = test_images.reshape((131, 28, 28, 1))

In [8]:
print('Training Dataset Shape:{} '.format(train_images.shape)) 
print('No. of Training Dataset Labels:{} '.format(len(train_labels)))
print('Validation Dataset Shape:{} '.format(validation_images.shape))
print('No. of Validation Dataset Labels:{} '.format(len(validation_labels)))
print('Test Dataset Shape:{} '.format(test_images.shape))
print('No. of Test Dataset Labels:{} '.format(len(test_labels)))

Training Dataset Shape:(1818, 28, 28, 1) 
No. of Training Dataset Labels:1818 
Validation Dataset Shape:(375, 28, 28, 1) 
No. of Validation Dataset Labels:375 
Test Dataset Shape:(131, 28, 28, 1) 
No. of Test Dataset Labels:131 


# CNN Model

In [19]:
cnn_model = ks.models.Sequential()
cnn_model.add(ks.layers.Conv2D(64, (5,5), activation= 'relu', input_shape= (28, 28, 1), name='Convolutional_layer_1'))
cnn_model.add(ks.layers.Conv2D(64, (5,5), activation= 'relu', name='Convolutional_layer_2'))
cnn_model.add(ks.layers.Conv2D(128, (3,3), activation= 'relu', name='Convolutional_layer_3'))

cnn_model.add(ks.layers.MaxPooling2D((2,2), name= 'Maxpooling_2D_1'))
cnn_model.add(ks.layers.MaxPooling2D((2,2), name= 'Maxpooling_2D_2'))
cnn_model.add(ks.layers.MaxPooling2D((2,2), name= 'Maxpooling_2D_3'))

cnn_model.add(ks.layers.Flatten(name= 'Flatten'))
cnn_model.add(ks.layers.Dense(128, activation='relu', name='Hidden_layer'))
cnn_model.add(ks.layers.Dense(10, activation='softmax', name='Output_layer'))

In [10]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convolutional_layer_1 (Conv2 (None, 24, 24, 64)        1664      
_________________________________________________________________
Convolutional_layer_2 (Conv2 (None, 20, 20, 64)        102464    
_________________________________________________________________
Convolutional_layer_3 (Conv2 (None, 18, 18, 128)       73856     
_________________________________________________________________
Maxpooling_2D_1 (MaxPooling2 (None, 9, 9, 128)         0         
_________________________________________________________________
Maxpooling_2D_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
Maxpooling_2D_3 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
Flatten (Flatten)            (None, 512)               0

In [11]:
cnn_model.compile(optimizer= 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
cnn_model.fit(train_images, train_labels, epochs= 40, validation_data= (validation_images, validation_labels))

Train on 1818 samples, validate on 375 samples
Epoch 1/40
1818/1818 [==============================] - 10s 6ms/sample - loss: 1.9384 - acc: 0.2327 - val_loss: 1.6281 - val_acc: 0.3387
Epoch 2/40
1818/1818 [==============================] - 10s 5ms/sample - loss: 1.4090 - acc: 0.4334 - val_loss: 1.3926 - val_acc: 0.4667
Epoch 3/40
1818/1818 [==============================] - 10s 6ms/sample - loss: 1.1766 - acc: 0.5138 - val_loss: 1.3134 - val_acc: 0.4880
Epoch 4/40
1818/1818 [==============================] - 10s 6ms/sample - loss: 1.0712 - acc: 0.5622 - val_loss: 1.2774 - val_acc: 0.5120
Epoch 5/40
1818/1818 [==============================] - 10s 5ms/sample - loss: 0.9766 - acc: 0.6056 - val_loss: 1.2019 - val_acc: 0.5947
Epoch 6/40
1818/1818 [==============================] - 10s 5ms/sample - loss: 0.9475 - acc: 0.6309 - val_loss: 1.1132 - val_acc: 0.5920
Epoch 7/40
1818/1818 [==============================] - 10s 5ms/sample - loss: 0.7979 - acc: 0.6947 - val_loss: 1.1360 - val_acc: 0

## Calculating Accuracies

In [13]:
train_loss, train_accuracy = cnn_model.evaluate(train_images, train_labels) 
print('Train Accuracy{} '.format(round(float(train_accuracy), 2)))

1818/1818 [==============================] - 2s 1ms/sample - loss: 0.1093 - acc: 0.9609
Train Accuracy0.96 


In [14]:
validation_loss, validation_accuracy = cnn_model.evaluate(validation_images, validation_labels) 
print('Validation Accuracy{} '.format(round(float(validation_accuracy), 2)))

375/375 [==============================] - 1s 1ms/sample - loss: 1.2796 - acc: 0.7813
Validation Accuracy0.78 


In [15]:
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels)
print('Test Accuracy {} '.format(round(float(test_accuracy), 2)))

131/131 [==============================] - 0s 1ms/sample - loss: 1.2120 - acc: 0.7252
Test Accuracy 0.73 


## Class Prediction

In [16]:
def predict_image(img, display1, display2):
    """
    Reads an image from testing dataset, and predict its label.
    
    Returns the image label and testing accuracy. 
    """
    
    img = np.array([img])

    img =img.reshape(1, 28, 28, 1)
    predictions = cnn_model.predict(img)
    score = tf.nn.softmax(predictions[0])
    
    display1.setText("This image most likely belongs to {}".format(class_names[np.argmax(predictions)]))
    display2.setText("Test Accuracy: {} %".format(round(float(test_accuracy * 100), 2)))

# GUI

In [17]:
class ApplicationWindow(QtWidgets.QMainWindow):
    def __init__(self):

        super(ApplicationWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        
        self.ui.browse_pushButton.clicked.connect(lambda: self.browse(self.ui.org_img_disp_label))
        
        
    def browse(self, Ui_MainWindow):
        """
        Uploads an image, resizes it for prediction,
        then resizes it again for plotting. 
        """

        filename = QtWidgets.QFileDialog.getOpenFileName(self, 'Open file', "*.tif")
        if (filename == ([], '')) | (filename ==  0 ):
            return
        imgPath = str(filename[0])
        try:
            img = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
            img_test = cv2.resize(img, (28, 28))
            img_show = cv2.resize(img, (721, 401))

        except Exception as e:
            print(' ')

        self.showImage(img_show, Ui_MainWindow)
        predict_image(img_test, self.ui.result_label, self.ui.result_label_2)
        
        
    def showImage(self, arr, Ui_MainWindow):
        """
        Convert an array to image and display it given image handler
        """

        imgBack = Image.fromarray(arr)
        qimage = PIL.ImageQt.ImageQt(imgBack)
        pixmap = QtGui.QPixmap.fromImage(qimage) 
        Ui_MainWindow.setPixmap(pixmap)
        Ui_MainWindow.setMask(pixmap.mask())
        Ui_MainWindow.show()

In [18]:
def main():
    app = QtWidgets.QApplication(sys.argv)
    application = ApplicationWindow()
    application.show()
    app.exec_()

if __name__ == "__main__":
    main()